In [112]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [37]:
def show(t):  # tensor
    print("size: ", t.size())
    print("value: ", t)
    print("requires_grad: ", t.requires_grad)
    print("grad_fn: ", t.grad_fn)
    print("grad: ", t.grad)

# Define the Net class

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## Define operations in NN, weights initialized by default
        # 1 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(1, 6, 5)  # conv1.weight and conv1.bias
        # 6 input image channel, 16 output channels, 5x5 square convolution
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # input channels 16 * 5 * 5, output 120 channels
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

In [16]:
def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension (the 1st dimension)
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

Net.num_flat_features = num_flat_features

In [17]:
def forward(self, x):  # Should be overridden by all subclasses of nn.Module
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # maxpooling
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)  # a simpler way of maxpooling
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

Net.forward = forward

# Initialization

In [20]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [102]:
params = list(net.parameters())
print(len(params))
for param in params:
    print(param.size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


# Forward Pass

In [126]:
input = torch.randn(1, 1, 32, 32)
out = net(input) # or, net.forward(input)
show(out)

size:  torch.Size([1, 10])
value:  tensor([[-0.0867, -0.1328,  0.1416, -0.0613, -0.0225,  0.1128,  0.0141,
          0.0759,  0.1292,  0.0332]])
requires_grad:  True
grad_fn:  <AddmmBackward object at 0x1089762e8>
grad:  None


In [127]:
target = torch.randn_like(out) # a dummy target
criterion = nn.MSELoss()  # define a loss function
loss = criterion(out, target)
show(loss)

size:  torch.Size([])
value:  tensor(0.6324)
requires_grad:  True
grad_fn:  <MseLossBackward object at 0x108976e10>
grad:  None


In [128]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# A Single Step of Backward Prop

In [129]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias before backward')
show(net.conv1.bias)

loss.backward()  # compute grad of net.parameters

print('conv1.bias after backward\n')
show(net.conv1.bias)

conv1.bias before backward
size:  torch.Size([6])
value:  Parameter containing:
tensor([ 0.1054, -0.0448,  0.0781,  0.1496, -0.0436, -0.0908])
requires_grad:  True
grad_fn:  None
grad:  tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias after backward

size:  torch.Size([6])
value:  Parameter containing:
tensor([ 0.1054, -0.0448,  0.0781,  0.1496, -0.0436, -0.0908])
requires_grad:  True
grad_fn:  None
grad:  tensor(1.00000e-02 *
       [ 1.2413,  1.2971,  0.6463,  0.2419,  0.7958,  0.0103])


# Loop: Update the Parameters by BP

In [130]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

for _ in range(3):  # in your training loop:
    optimizer.zero_grad()   # zero the gradient buffers of all parameters
    output = net(input)
    loss = criterion(output, target)
    loss.backward()     # compute grad of net.parameters   
    optimizer.step()    # Does the update on weights/bias by their grad

In [ ]:
tor